In [81]:
import os

In [82]:
os.environ['OPENAI_API_KEY'] = 'deleted_key_before_being_deleted_by_huggingface'
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [83]:
from langchain.chat_models import ChatOpenAI
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [84]:
# load from disk
from langchain.vectorstores import Chroma
db = Chroma(
    persist_directory="../data/docs/chroma_cos", 
    embedding_function=embedding
)

In [85]:
##### Conversational Retrieval #####
from langchain.agents.agent_toolkits.conversational_retrieval.tool import (
    create_retriever_tool,
)
retriever = db.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "document-retriever",
    "Query a retriever to get information about the video game dataset.",
)
##################################

In [86]:
from typing import List
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


class Response(BaseModel):
    """Final response to the question being asked.
        If you do not have an answer, say you do not have an answer, and ask the user to ask another recommendation.
        If you do have an answer, be verbose and explain why you think the game answers the user's query.
        Don't give information not mentioned in the documents CONTEXT.
        You should always refuse to answer questions that are not related to this specific domain, of video game recommendation.
        If no document passes the minimum threshold of similarity .75, default to apologizing for no answer.
    """

    answer: str = Field(description="The final answer to the user, including the names in the answer.")
    name: List[str] = Field(
        description="A list of the names of the games found for the user. Only include the game name if it was given as a result to the user's query."
    )

In [87]:
from langchain.schema.agent import AgentActionMessageLog, AgentFinish
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [88]:
from langchain.schema.agent import AgentActionMessageLog, AgentFinish
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [89]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function

In [90]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a recommendation assistant, based off documents."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [91]:
llm_with_tools = llm.bind(
    functions=[
        # The retriever tool
        format_tool_to_openai_function(retriever_tool),
        # Response schema
        convert_pydantic_to_openai_function(Response),
    ]
)

In [92]:
agent = (
    {
        "input": lambda x: x["input"],
        # Format agent scratchpad from intermediate steps
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | parse
)

In [93]:
agent_executor = AgentExecutor(
    tools=[retriever_tool], 
    agent=agent, 
    verbose=True
)

In [94]:
post_prompt = """
    1. Respond with a respectable and friendy tone.
    2. You should give the best possible answer based on user's query. 
    3. Do not give me any information that is not included in the document. 
    4. If you are able to, provide the links to the steam site for the games answer.
    5. If you need more context from the user, ask them to provide more context in the next query. Do not include games that contain the queried game in the title.
    6. If a user asks for a type of game, use that type to find a game that mentions the type.
"""
# If you do not have an answer, your response should be kind and apologetic, as to why you do not have an answer. 
# If a user asks for a specific number of games, and you cannot provide that, answer with what games you found and explain why you could not find others.

In [95]:
# prompt = "What game is similar to Counter-Strike:global offensive?"
prompt = "Give me 2 action games with robots?"
# prompt = "What game has animals?"
# prompt = "What is a game similar to CSGO, besides CSGO"
# prompt = "What is a game similar to CSGO?"

In [96]:
assistant_response = ""
docs = agent_executor.invoke(
                {"input": f"{prompt} {post_prompt}"},
                return_only_outputs=True,
            )                                     # retrieve response from chatgpt
try:
    assistant_response += docs["answer"]
except:
    assistant_response += docs["output"]



> Entering new AgentExecutor chain...
[Document(page_content='steam-driven robots', metadata={'name': 'SteamWorld Heist'}), Document(page_content='game-esque minigame with two robots fighting each other)', metadata={'name': 'Thrillville®: Off the Rails™'}), Document(page_content='robots.', metadata={'name': 'The Uncertain: Last Quiet Day'}), Document(page_content='robots.', metadata={'name': 'Rapid_Fire'})]{'arguments': '{\n  "answer": "I found two action games with robots for you: SteamWorld Heist and Thrillville®: Off the Rails™. SteamWorld Heist is a turn-based strategy game where you control a team of steam-driven robots. You can find more information about SteamWorld Heist [here](https://store.steampowered.com/app/322190/SteamWorld_Heist/). Thrillville®: Off the Rails™ is a game that features a game-esque minigame with two robots fighting each other. You can find more information about Thrillville®: Off the Rails™ [here](https://store.steampowered.com/app/15120/Thrillville_Off_t

In [97]:
docs['answer']

'I found two action games with robots for you: SteamWorld Heist and Thrillville®: Off the Rails™. SteamWorld Heist is a turn-based strategy game where you control a team of steam-driven robots. You can find more information about SteamWorld Heist [here](https://store.steampowered.com/app/322190/SteamWorld_Heist/). Thrillville®: Off the Rails™ is a game that features a game-esque minigame with two robots fighting each other. You can find more information about Thrillville®: Off the Rails™ [here](https://store.steampowered.com/app/15120/Thrillville_Off_the_Rails/).'

In [98]:
# Which we could store and use them on our plotting page
print(len(docs['name']))
print(docs['name'])

2
['SteamWorld Heist', 'Thrillville®: Off the Rails™']
